In [1]:
import sys
import os
import logging
sys.path.append(r"..")
from utils import *
import numpy as np
import pandas as pd
import deepctr
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from gensim import models
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix

from collections import defaultdict
from collections import Counter

In [2]:
user_ids = pd.read_pickle(f"{pickle_path}/user_ids.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_ids = user_ids.merge(user, how='left', on='user_id')
del user
print(user_ids.shape)
user_ids.head()

(1900000, 16)


,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,creative_id_len,ad_id_len,product_id_len,product_category_len,advertiser_len,industry_len,age,gender
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169...","[724607, 188507, 773445, 1458878, 109959, 6621...","[0, 136, 0, 0, 1334, 0, 0, 0, 0, 1454, 64, 64,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[7293, 9702, 29455, 14668, 11411, 14681, 17189...","[326, 6, 106, 326, 0, 326, 73, 217, 64, 238, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]",12,12,6,3,12,9,4.0,1.0
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ...","[58788, 139702, 38066, 541125, 14495, 392680, ...","[87, 80, 129, 129, 1400, 87, 0, 1261, 0, 0, 11...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[22885, 10686, 18562, 25932, 768, 22885, 34505...","[318, 238, 6, 6, 317, 318, 47, 6, 47, 47, 242,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",42,42,20,3,36,15,10.0,1.0
2,3,"[661347, 808612, 710859, 825434, 593522, 72694...","[586668, 713448, 629278, 728308, 527601, 64310...","[36256, 40905, 1674, 35985, 1674, 0, 0, 1031, ...","[17, 17, 2, 17, 2, 18, 18, 2, 2, 2, 18, 12, 2,...","[32974, 9877, 18492, 14186, 17018, 9058, 8371,...","[0, 0, 322, 0, 322, 6, 54, 6, 322, 322, 205, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",30,30,17,6,28,8,7.0,2.0
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264...","[37966, 524312, 511235, 1638619, 1698206, 1953...","[1862, 0, 2625, 38743, 41265, 37758, 39904, 39...","[2, 18, 2, 17, 17, 4, 17, 17, 18, 2, 2, 2, 2, ...","[19451, 7976, 13084, 12130, 23664, 811, 10172,...","[238, 25, 248, 0, 0, 0, 0, 0, 88, 319, 238, 6,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",29,29,18,6,26,10,5.0,1.0
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...","[265971, 314795, 24966, 41148, 751113, 925792,...","[0, 0, 87, 136, 0, 136, 0, 0, 0, 1064, 0, 2620...","[5, 8, 2, 2, 18, 2, 5, 18, 18, 2, 18, 2, 5, 5,...","[11882, 992, 22885, 9706, 38760, 2862, 17745, ...","[297, 0, 318, 6, 322, 6, 288, 322, 319, 238, 3...","[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",33,33,7,4,30,18,4.0,1.0


In [19]:
## 计算idf值 creative_id
import itertools
from collections import Counter
def fea_idf(fea):
    fea_id_lists = user_ids[fea].map(lambda x: list(set(x)))
    fea_l = list(itertools.chain(*fea_id_lists))
    save_pickle(Counter(fea_l), pickle_path+f'{fea}_idf')
    
fea_idf('creative_id')  
fea_idf('ad_id')  
fea_idf('product_id')  
fea_idf('product_category')  
fea_idf('advertiser_id')  
fea_idf('industry')  
fea_idf('click_times')  

In [3]:
def tfidf_matrix_p(features_list, min_dfs):
    for fea, min_df in zip(features_list, min_dfs):
        tfidfer = TfidfVectorizer(analyzer='word', token_pattern=u"(?u)\\b\\w+\\b", min_df=min_df)  # (?u)\b\w+\b"，这样就不会忽略单个的字符
        if fea == 'creative_id':
            tfidf_matrix = tfidfer.fit_transform(user_ids[fea].map(lambda x:' '.join([str(i) for i in x])).values)
            save_pickle(tfidfer.vocabulary_, pickle_path+f'creative_id_tfidf_v_{min_df}')
            save_pickle(tfidf_matrix, pickle_path+f'creative_id_tfidf_m_{min_df}')

            print(f'tfidf matrix {fea} size:{tfidf_matrix.shape}')
        else:
            tfidf_matrix_fea = tfidfer.fit_transform(user_ids[fea].map(lambda x:' '.join([str(i) for i in x])).values)
            print(f'tfidf matrix {fea} size:{tfidf_matrix_fea.shape}')
            tfidf_matrix = csr_matrix(sparse.hstack((tfidf_matrix, tfidf_matrix_fea)))
            save_pickle(tfidfer.vocabulary_, pickle_path+f'{fea}_tfidf_v_{min_df}')
            save_pickle(tfidf_matrix_fea, pickle_path+f'{fea}_tfidf_m_{min_df}')
    
    save_pickle(tfidf_matrix, pickle_path+f'tfidf_m')
   
    print(f'tfidf matrix size:{tfidf_matrix.shape}')
    return tfidf_matrix

In [4]:
tfidf_features = ['creative_id', 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry', 'click_times']
max_features = [50, 50, 5, 5, 5, 5, 5]
tfidf_matrix = tfidf_matrix_p(tfidf_features, max_features)

tfidf matrix creative_id size:(1900000, 138554)
tfidf matrix ad_id size:(1900000, 138889)
tfidf matrix product_id size:(1900000, 22953)
tfidf matrix product_category size:(1900000, 18)
tfidf matrix advertiser_id size:(1900000, 40950)
tfidf matrix industry size:(1900000, 317)
tfidf matrix click_times size:(1900000, 8)
tfidf matrix size:(1900000, 341689)


In [6]:
offline = False
if offline:
    train_filter = 700000
else:
    train_filter = 900000


In [9]:
def lgb_class(pred_y, num_class):    
    lgb_params = {  
        'boosting_type': 'gbdt',  
        'objective': 'multiclass',  
        'num_class': num_class,  
        'metric': 'multi_error',  
        'num_leaves': 63,  
        'learning_rate': 0.05,  
        'feature_fraction': 0.8,  
        'bagging_fraction': 0.8,  
        'bagging_freq': 5,  
        'lambda_l1': 1,  
        'lambda_l2': 5,  
        'min_gain_to_split': 0.2,  
        'verbose': -1, 
        'n_jobs':10
    }
    
    
    train_filter = 900000
    train_x = tfidf_matrix[0:train_filter]
    train_y = (user_ids[pred_y]-1)[0:train_filter].astype(np.int)
    test_x =  tfidf_matrix[900000:]


    n_fold = 5
    result = np.zeros((test_x.shape[0],2))
    oof = np.zeros((train_x.shape[0],2))
    skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=2019)
    kfold = skf.split(train_x, train_y)
    count_fold = 0
    for train_index, vali_index in kfold:
        print("training......fold",count_fold)
        count_fold = count_fold + 1
        k_x_train = train_x[train_index]
        k_y_train = train_y[train_index]
        k_x_vali = train_x[vali_index]
        k_y_vali = train_y[vali_index]


        trn = lgb.Dataset(k_x_train, k_y_train)
        val = lgb.Dataset(k_x_vali, k_y_vali)
        lgb_model = lgb.train(lgb_params, train_set=trn, valid_sets=[trn, val],
                      num_boost_round=4000,early_stopping_rounds=200, verbose_eval=50)
        test_pred_proba = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration)
        val_pred_proba = lgb_model.predict(k_x_vali, num_iteration=lgb_model.best_iteration)

        result = result + test_pred_proba
        oof[vali_index] = val_pred_proba
    return result, oof

gender_clf = MultinomialNB().fit(tfidf_matrix[0:train_filter], user_ids['gender'][0:train_filter])
age_clf = MultinomialNB().fit(tfidf_matrix[0:train_filter], user_ids['age'][0:train_filter])
if offline:
    train_filter = 700000
    gender_p = gender_clf.predict(tfidf_matrix[train_filter:900000])
    age_p = age_clf.predict(tfidf_matrix[train_filter:900000])
else:
    train_filter = 900000
    gender_p = gender_clf.predict(tfidf_matrix[train_filter:])
    age_p = age_clf.predict(tfidf_matrix[train_filter:])

In [10]:
gender_pre, gender_oof = lgb_class('gender',2)


training......fold 0
Training until validation scores don't improve for 200 rounds.
[50]	training's multi_error: 0.131386	valid_1's multi_error: 0.131439
[100]	training's multi_error: 0.110399	valid_1's multi_error: 0.111661
[150]	training's multi_error: 0.0992306	valid_1's multi_error: 0.101417
[200]	training's multi_error: 0.0923944	valid_1's multi_error: 0.0951611


KeyboardInterrupt: 

评估提交

In [43]:
if offline:
    gender_accuracy = accuracy_score(user_ids['gender'][train_filter:900000], gender_p)
    age_accuracy = accuracy_score(user_ids['age'][train_filter:900000], age_p)
    print(gender_accuracy, age_accuracy, gender_accuracy+age_accuracy)
else:
    train_gender_p = gender_clf.predict(data_tfidf_gender[:train_filter])
    train_age_p = age_clf.predict(data_tfidf_age[:train_filter])
    
    gender_accuracy = accuracy_score(user_creative_ids['gender'][:train_filter], train_gender_p)
    age_accuracy = accuracy_score(user_creative_ids['age'][:train_filter], train_age_p)
    print(gender_accuracy, age_accuracy, gender_accuracy+age_accuracy)
    sub = pd.DataFrame()
    sub['user_id'] = range(3000001,4000001)
    sub['predicted_age'] = age_p
    sub['predicted_gender'] = gender_p
    sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')
    print(sub.shape)
    sub.head()

0.91127 0.38208 1.29335


0.861665 0.280975 1.14264  
0.90131 0.31792 1.21923  min_df=5  
0.909795 0.35963 1.269425  min_df=25    (1900000, 262005)  
0.908905 0.367675 1.27658  min_df=50  (1900000, 138555)  
0.905905 0.36898 1.2748849999999998  100 (1900000, 71208)  
0.909795 0.36898 1.278775  线上1.285234  25和100的结合    


0.91127 0.38208 1.29335



In [31]:
lr = LogisticRegression(solver='lbfgs', multi_class='ovr', n_jobs=4)
lr_model = lr.fit(tfidf_matrix[0:train_filter], user_ids['gender'][0:train_filter])
lr_pred = lr_model.predict(tfidf_matrix[train_filter:900000])

In [33]:
lr_pred_r = np.rint(lr_pred)
lr_pred_r = np.clip(lr_pred_r, 1, 2)
gender_accuracy = accuracy_score(user_ids['gender'][train_filter:900000], lr_pred_r)
gender_accuracy,lr_pred_r.shape

(0.9222, (200000,))

(0.922435,  0.418795  
0.92264  0.41987  

In [39]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'n_estimators': 5000,
    'metric': 'mae',
    'learning_rate': 0.01,
    'min_child_samples': 5,
    'min_child_weight': 0.01,
    'subsample_freq': 1,
    'num_leaves': 31,
    'max_depth': -1,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'reg_alpha': 0,
    'reg_lambda': 5,
    'verbose': -1,
    'random_state': 4590,
    'n_jobs': 4,
}

In [40]:
def lr_pred(pred_y):
    cv = True
    model_type = 'lgb'
    seed = 2020
    train_filter = 900000
    train_x = tfidf_matrix[0:train_filter]
    train_y = user_ids[pred_y][0:train_filter]
    test_x =  tfidf_matrix[900000:]
    if cv:
        n_fold = 5
        count_fold = 0
        preds_list = list()
        oof = np.zeros(train_x.shape[0])
        kfolder = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
        kfold = kfolder.split(train_x, train_y)
        for train_index, vali_index in kfold:
            print("training......fold",count_fold)
            count_fold = count_fold + 1
            k_x_train = train_x[train_index]
            k_y_train = train_y[train_index]
            k_x_vali = train_x[vali_index]
            k_y_vali = train_y[vali_index]
            if model_type == 'lgb':
                reg = LogisticRegression(solver='lbfgs', multi_class='multinomial', n_jobs=-1)
                reg_model = reg.fit(k_x_train, k_y_train)
                k_pred = reg_model.predict(k_x_vali)
                pred = reg_model.predict(test_x)
    #             lgb_model = lgb_model.fit(k_x_train, k_y_train, eval_set=[(k_x_vali, k_y_vali)],
    #                                       early_stopping_rounds=200, verbose=False, eval_metric="mae",
    #                                       )
    #             k_pred = lgb_model.predict(k_x_vali, num_iteration=lgb_model.best_iteration_)
    #             pred = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration_)
            preds_list.append(pred)
            oof[vali_index] = k_pred
    preds_columns = ['preds_{id}'.format(id=i) for i in range(n_fold)]
    preds_df = pd.DataFrame(data=preds_list)
    preds_df = preds_df.T
    preds_df.columns = preds_columns
    preds = list(preds_df.mean(axis=1))
    return preds, oof

In [41]:
gender_pre, gender_oof = lr_pred('gender')
age_pre, age_oof = lr_pred('age')


training......fold 0
training......fold 1
training......fold 2
training......fold 3
training......fold 4
training......fold 0
training......fold 1
training......fold 2
training......fold 3
training......fold 4


In [45]:
preds22 = np.rint(gender_oof)
preds22 = np.clip(preds22, 1, 2)
gender_accuracy = accuracy_score(user_ids['gender'][0:900000], preds22)
gender_accuracy,preds22.shape

(0.9214966666666666, (900000,))

In [50]:
gender_pre2, gender_oof2 = np.clip(np.rint(gender_pre), 1, 2),np.clip(np.rint(gender_oof), 1, 2)
age_pre2, age_oof2 = np.clip(np.rint(age_pre), 1, 10),np.clip(np.rint(age_oof), 1, 10)


gender_accuracy = accuracy_score(user_ids['gender'][:900000], gender_oof2)
age_accuracy = accuracy_score(user_ids['age'][:900000], age_oof2)
print(gender_accuracy, age_accuracy, gender_accuracy+age_accuracy)
sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = age_pre2
sub['predicted_gender'] = gender_pre2
sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')
print(sub.shape)
sub.head()

0.9222855555555556 0.42004555555555556 1.3423311111111111
(1000000, 3)


,user_id,predicted_age,predicted_gender
0,3000001,3.0,1.0
1,3000002,7.0,2.0
2,3000003,4.0,1.0
3,3000004,3.0,1.0
4,3000005,3.0,1.0


0.9222855555555556 0.42004555555555556 1.3423311111111111  
线上1.3472